In [ ]:
import os
import shutil

from support_function import *
from scipy.io import loadmat
import numpy as np

In [ ]:
def get_img_and_corresponding_bbx(path):

    folder_list = []
    for element in os.walk(path): folder_list.append(element)
    del folder_list[0]
        
    boxes_list = []
    for folder in folder_list:
        boxes = np.squeeze(loadmat(folder[0] + "/" + 'polygons.mat')['polygons'])
        prepended_name = folder[0].split('/')[-1].lower()
        boxes = getListBoxes(boxes)
        folder[2].sort()
        for i, photo_name in zip(range(100), folder[2]):

            source_img_path = folder[0] + "/" + photo_name
            image = cv2.imread(source_img_path)
            height, width, _ = image.shape
            # print(height, width)
            new_dest_name = f'{prepended_name}_img_{i}.jpg'
            dest_img_dir = "../../utils/datasets/egohands/train/images"
            dest_img_path = os.path.join(dest_img_dir, new_dest_name)
            bbxs = getBoxes(i, boxes)

            shutil.copy(source_img_path, dest_img_path)
            # [x y width height] is the bbox format found in the README and what is in bbxs
            for bbx in bbxs:

                bbox_width = bbx[1][0] - bbx[0][0]
                bbox_height = bbx[1][1] - bbx[0][1]

                x_center = bbx[0][0] + (bbox_width / 2)
                y_center = bbx[0][1] + (bbox_height / 2)
                
                
                with open(dest_img_path.replace("images", "labels").replace(".jpg", ".txt"), 'a') as f:
                    f.write(f'0 {x_center/width} {y_center/height} {bbox_width/width} {bbox_height/height}\n')
            
            print()
            
            boxes_list.append(getBoxes(i, boxes))
        # break

In [ ]:

#path to egohands_data/_LABELLED_SAMPLES folder

root = os.path.dirname(os.path.abspath(os.getcwd()))

get_img_and_corresponding_bbx(path=root + '/egohands_data/_LABELLED_SAMPLES')